In [1]:
import asyncio
import pandas as pd
from pyinjective.async_client import AsyncClient
from pyinjective.client.model.pagination import PaginationOption
from pyinjective.core.network import Network
from datetime import datetime, timedelta


# Initialize the network and client
network = Network.local()
client = AsyncClient(network)

async def fetch_derivative_orders(subaccount_id, market_ids, start_date, end_date, csv_name, client):
    # Convert start and end dates to milliseconds since epoch
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)

    orders_list = []
    skip = 0
    limit = 100  # Set the number of results to fetch per request
    print("test")
    while True:
        pagination = PaginationOption(skip=skip, limit=limit)
        print("test")
        # Fetch the orders history
        orders_response = await client.fetch_derivative_orders_history(
            subaccount_id=subaccount_id,
            market_ids=market_ids,
            pagination=pagination,
        )
        print(orders_response)
        # orders_response = await client.fetch_spot_orders_history(
        #     subaccount_id=subaccount_id,
        #     #market_ids=market_ids,
        #     # is_conditional="false",
        #     pagination=pagination,
        # )
        
        
        # Break the loop if no orders were returned
        if not orders_response['orders']:
            break

        # Iterate through each order
        for order in orders_response['orders']:
            created_at = pd.to_datetime(order['createdAt'], unit='ms')
            updated_at = pd.to_datetime(order['updatedAt'], unit='ms')
            
            
            # Stop if we've gone past the start date

            start_timestamp = pd.to_datetime(start_timestamp, unit='ms')

            print (created_at)
            print (start_timestamp)


            if created_at < start_timestamp:
                # Save orders to CSV and return
                df_orders = pd.DataFrame(orders_list)
                df_orders.to_csv(csv_name, index=False)
                return df_orders
            
            # Generate the time range between createdAt and updatedAt
            
            # For each second in the time range, add a new row to the data
    
            orders_list.append({
                'orderHash': order['orderHash'],
                'marketId': order['marketId'],
                'subaccountId': order['subaccountId'],
                'executionType': order['executionType'],
                'orderType': order['orderType'],
                'price': order['price'],
                'quantity': order['quantity'],
                'updated_at': updated_at,
                'state': order['state'],
                'created_at': created_at,
            })

        # Increment skip for pagination
        skip += limit

    # Save remaining orders to CSV
    df_orders = pd.DataFrame(orders_list)
    df_orders.to_csv(csv_name, index=False)

    return df_orders

async def main():
    subaccount_id = "0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000001"

    # subaccount_id = "0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000000f" # tao
    # subaccount_id = "0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000013" # stx
    # subaccount_id = '0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000012' # ftm
    # subaccount_id = '0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000011' # pepe
    # subaccount_id = '0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000006' # crv
    # subaccount_id = '0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000010' # popcat
    # subaccount_id = '0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000005' # mkr

    #subaccount_id = '0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000019' # fet

    #subaccount_id = '0x935b705b5943b0e1b859651d2ccf375bc96dfdd700000000000000000000001d'  # hype

    #subaccount_id = '0xed8c4c43e03e24b7f12975472da771ce2f8b857c000000000000000000000000'

    # subaccount_id = '0xadbce46487a056a56b8819600add44efbd599d57000000000000000000000001' # 26nf

    
    market_ids = [
                    # "0xa51b0df7a1ac2bea408fdc4f84f3d7060b62f976ab659b329b00065e5c8f2c37" # tao
                    # '0x9bb2341e12d7357cd6865b69e491bab5ac8b087df50906a53ffa8fc3ede59f5f' # stx
                    # '0x2d7092545081b81ba33bf817b302f9609254f15f4354016631aec3bb39461f99' # "pepe
                    # '0x8157dd4bf502fc688aaa722c725124da3f411d7a92c569d55f34826f9ee040a9' # popcat
                    # '0x4be4791338907626dd77a806c6e4dff76d1428768080fe232f32ef990c8d064f'  # ftm
                    # '0xf58079e67f845907e93ab9767126a226a0759c23bee1bfc880fa8fba98f25872' # crv
                    # "0x142d0fa4506b5f404bcfdd54567797ff6767dce07afaedc90d379665f09f0520" # mkr
 
                    # '0xc9030edef611568ec9aa48228c92e30d398abf0eb289b5fee873b0f2f3a80295' # fet spot

                    '0x5a9c0f24d6bdd55373ec3d20bfb5c041674bab183dfff0548d1c8fedbf243ca1' # hype perp
    ]
    
    

    # start_date = "2024-10-02 10:00:00"  # Start date for fetching orders
    # end_date = "2024-10-02 1:00:00"    # End date for fetching orders


    now = datetime.now()

    start_date = (now - timedelta(hours=1)).strftime('%Y-%m-%d %H:%M:%S')
    end_date = now.strftime('%Y-%m-%d %H:%M:%S')
    csv_name = f"csv/test.csv"

    # Call the fetch_derivative_orders function
    df_orders = await fetch_derivative_orders(subaccount_id, market_ids, start_date, end_date, csv_name, client)
    print("Orders saved to CSV:", csv_name)

await main()


/Users/rvie/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


test
test
{'paging': {'total': '0', 'from': 0, 'to': 0, 'countBySubaccount': '0', 'next': []}, 'orders': []}
Orders saved to CSV: csv/test.csv


In [2]:
import asyncio
import pandas as pd
from pyinjective.async_client import AsyncClient
from pyinjective.client.model.pagination import PaginationOption
from pyinjective.core.network import Network
from datetime import datetime, timedelta
from functions import marketId_to_ticker

# Initialize the network and client
network = Network.local()
client = AsyncClient(network)

async def fetch_derivative_orders(subaccount_id, market_ids, start_date, end_date, client):
    # Convert start and end dates to milliseconds since epoch
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)

    orders_list = []
    skip = 0
    limit = 100  # Set the number of results to fetch per request

    while True:
        pagination = PaginationOption(skip=skip, limit=limit)

        # Fetch the orders history
        orders_response = await client.fetch_derivative_orders_history(
            subaccount_id=subaccount_id,
            market_ids=market_ids,
            pagination=pagination,
        )

        # Break the loop if no orders were returned
        if not orders_response['orders']:
            break

        # Iterate through each order
        for order in orders_response['orders']:
            created_at = pd.to_datetime(order['createdAt'], unit='ms')
            updated_at = pd.to_datetime(order['updatedAt'], unit='ms')

            if created_at < pd.to_datetime(start_timestamp, unit='ms'):
                return orders_list

            orders_list.append({
                'orderHash': order['orderHash'],
                'marketId': order['marketId'],
                'subaccountId': order['subaccountId'],
                'executionType': order['executionType'],
                'orderType': order['orderType'],
                'price': order['price'],
                'quantity': order['quantity'],
                'updated_at': updated_at,
                'state': order['state'],
                'created_at': created_at,
            })

        # Increment skip for pagination
        skip += limit

    return orders_list

async def main():
    # List of subaccount IDs

    subaccount_id = "0xbcc871dab5a507624e55afeaa93610a424c446cc000000000000000000000000"
    ticker = "PEPE/USDT PERP"
    # Market IDs for fetching orders
 

    now = datetime.now()
    start_date = (now - timedelta(hours=1)).strftime('%Y-%m-%d %H:%M:%S')
    end_date = now.strftime('%Y-%m-%d %H:%M:%S')


    market_id = await marketId_to_ticker(ticker)
    # Fetch orders for each subaccount
    orders = await fetch_derivative_orders(subaccount_id, market_id, start_date, end_date, client)

    # Convert all orders to a DataFrame and save as a single CSV
    df_all_orders = pd.DataFrame(orders)
    csv_name = "csv/test.csv"
    df_all_orders.to_csv(csv_name, index=False)
    print(f"All orders saved to {csv_name}")

await main()

All orders saved to csv/test.csv
